## Wprowadzenie do regresji - Lekcja 1

#### Umieszczenie w kontekście

✅ Istnieje wiele rodzajów metod regresji, a wybór odpowiedniej zależy od pytania, na które chcesz odpowiedzieć. Jeśli chcesz przewidzieć prawdopodobny wzrost osoby w określonym wieku, użyjesz `regresji liniowej`, ponieważ szukasz **wartości numerycznej**. Jeśli interesuje Cię ustalenie, czy dany typ kuchni powinien być uznany za wegański, szukasz **przypisania kategorii**, więc użyjesz `regresji logistycznej`. Więcej o regresji logistycznej dowiesz się później. Zastanów się nad pytaniami, które możesz zadać danym, i które z tych metod byłyby bardziej odpowiednie.

W tej sekcji będziesz pracować z [małym zestawem danych dotyczącym cukrzycy](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Wyobraź sobie, że chcesz przetestować leczenie dla pacjentów z cukrzycą. Modele uczenia maszynowego mogą pomóc w określeniu, którzy pacjenci lepiej zareagują na leczenie, na podstawie kombinacji zmiennych. Nawet bardzo podstawowy model regresji, gdy zostanie zwizualizowany, może dostarczyć informacji o zmiennych, które pomogą Ci zorganizować teoretyczne badania kliniczne.

A zatem, zaczynajmy!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Ilustracja autorstwa @allison_horst</figcaption>

<!--![Ilustracja autorstwa \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.pl.jpg)<br>Ilustracja autorstwa @allison_horst-->


## 1. Ładowanie naszego zestawu narzędzi

Do tego zadania będziemy potrzebować następujących pakietów:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) to [zbiór pakietów R](https://www.tidyverse.org/packages) zaprojektowany, aby uczynić analizę danych szybszą, łatwiejszą i bardziej przyjemną!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) to [zbiór pakietów](https://www.tidymodels.org/packages/) przeznaczonych do modelowania i uczenia maszynowego.

Możesz je zainstalować za pomocą:

`install.packages(c("tidyverse", "tidymodels"))`

Poniższy skrypt sprawdza, czy masz zainstalowane pakiety potrzebne do ukończenia tego modułu, i instaluje je w przypadku, gdyby jakieś brakowały.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Teraz załadujmy te niesamowite pakiety i udostępnijmy je w naszej bieżącej sesji R. (To tylko ilustracja, `pacman::p_load()` już to dla Ciebie zrobił)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Zbiór danych dotyczących cukrzycy

W tym ćwiczeniu wykorzystamy nasze umiejętności regresji, aby przewidywać na podstawie zbioru danych dotyczących cukrzycy. [Zbiór danych dotyczących cukrzycy](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) zawiera `442 próbki` danych związanych z cukrzycą, z 10 zmiennymi predykcyjnymi: `wiek`, `płeć`, `wskaźnik masy ciała`, `średnie ciśnienie krwi` oraz `sześć pomiarów surowicy krwi`, a także zmienną wynikową `y`: ilościowy wskaźnik postępu choroby rok po badaniu bazowym.

|Liczba obserwacji|442|
|------------------|:---|
|Liczba predyktorów|Pierwsze 10 kolumn to numeryczne zmienne predykcyjne|
|Wynik/Cel|Kolumna 11 to ilościowy wskaźnik postępu choroby rok po badaniu bazowym|
|Informacje o predyktorach|- wiek w latach
||- płeć
||- bmi wskaźnik masy ciała
||- bp średnie ciśnienie krwi
||- s1 tc, całkowity cholesterol w surowicy
||- s2 ldl, lipoproteiny o niskiej gęstości
||- s3 hdl, lipoproteiny o wysokiej gęstości
||- s4 tch, całkowity cholesterol / HDL
||- s5 ltg, prawdopodobnie logarytm poziomu trójglicerydów w surowicy
||- s6 glu, poziom cukru we krwi|

> 🎓 Pamiętaj, że to uczenie nadzorowane, więc potrzebujemy określonego celu 'y'.

Zanim będziesz mógł manipulować danymi w R, musisz zaimportować dane do pamięci R lub zbudować połączenie z danymi, które R może wykorzystać do zdalnego dostępu.

> Pakiet [readr](https://readr.tidyverse.org/), który jest częścią Tidyverse, oferuje szybki i przyjazny sposób na wczytywanie danych prostokątnych do R.

Teraz załadujmy zbiór danych dotyczących cukrzycy z podanego adresu URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Dodatkowo przeprowadzimy kontrolę poprawności naszych danych za pomocą `glimpse()` i wyświetlimy pierwsze 5 wierszy za pomocą `slice()`.

Zanim przejdziemy dalej, wprowadźmy coś, co często spotkasz w kodzie R 🥁🥁: operator pipe `%>%`

Operator pipe (`%>%`) wykonuje operacje w logicznej kolejności, przekazując obiekt do funkcji lub wyrażenia. Możesz myśleć o operatorze pipe jako o powiedzeniu "a następnie" w swoim kodzie.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` pokazuje nam, że te dane mają 442 wiersze i 11 kolumn, przy czym wszystkie kolumny są typu danych `double`.

<br>

> glimpse() i slice() to funkcje w [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, będący częścią Tidyverse, to gramatyka manipulacji danymi, która oferuje spójny zestaw czasowników pomagających rozwiązywać najczęstsze wyzwania związane z manipulacją danymi.

<br>

Teraz, gdy mamy dane, skupmy się na jednej zmiennej (`bmi`), którą będziemy analizować w tym ćwiczeniu. Wymaga to wybrania odpowiednich kolumn. Jak to zrobić?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) pozwala nam *wybrać* (i opcjonalnie zmienić nazwy) kolumny w ramce danych.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Dane treningowe i testowe

W uczeniu nadzorowanym często stosuje się praktykę *dzielenia* danych na dwa podzbiory: (zazwyczaj większy) zestaw do trenowania modelu oraz mniejszy zestaw "odłożony na bok", który służy do oceny, jak model sobie poradził.

Teraz, gdy mamy gotowe dane, możemy sprawdzić, czy maszyna może pomóc w określeniu logicznego podziału liczb w tym zbiorze danych. Możemy skorzystać z pakietu [rsample](https://tidymodels.github.io/rsample/), który jest częścią frameworku Tidymodels, aby stworzyć obiekt zawierający informacje o tym, *jak* podzielić dane, a następnie użyć dwóch dodatkowych funkcji rsample do wyodrębnienia utworzonych zestawów treningowych i testowych:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Trenuj model regresji liniowej za pomocą Tidymodels

Teraz jesteśmy gotowi, aby wytrenować nasz model!

W Tidymodels modele definiuje się za pomocą `parsnip()`, określając trzy kluczowe elementy:

-   **Typ modelu** rozróżnia modele, takie jak regresja liniowa, regresja logistyczna, modele drzew decyzyjnych i inne.

-   **Tryb modelu** obejmuje popularne opcje, takie jak regresja i klasyfikacja; niektóre typy modeli obsługują oba te tryby, podczas gdy inne mają tylko jeden tryb.

-   **Silnik modelu** to narzędzie obliczeniowe, które będzie używane do dopasowania modelu. Często są to pakiety R, takie jak **`"lm"`** lub **`"ranger"`**

Te informacje o modelu są przechowywane w specyfikacji modelu, więc zbudujmy jedną!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Po określeniu modelu, można go `oszacować` lub `wytrenować` za pomocą funkcji [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), zazwyczaj używając formuły i danych.

`y ~ .` oznacza, że dopasujemy `y` jako przewidywaną wartość/cel, wyjaśnioną przez wszystkie predyktory/cechy, czyli `.` (w tym przypadku mamy tylko jeden predyktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Z wyników modelu możemy zobaczyć współczynniki wyuczone podczas treningu. Reprezentują one współczynniki linii najlepszego dopasowania, która daje nam najniższy ogólny błąd między rzeczywistą a przewidywaną zmienną.
<br>

## 5. Dokonaj prognoz na zbiorze testowym

Teraz, gdy wytrenowaliśmy model, możemy go użyć do przewidywania postępu choroby y dla zbioru danych testowych, korzystając z [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). To będzie użyte do narysowania linii między grupami danych.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Hurra! 💃🕺 Właśnie wytrenowaliśmy model i użyliśmy go do wykonania prognoz!

Podczas wykonywania prognoz konwencja tidymodels zakłada zawsze generowanie tibble/ramki danych z wynikami, które mają ustandaryzowane nazwy kolumn. Dzięki temu łatwo połączyć oryginalne dane z prognozami w użytecznym formacie do dalszych operacji, takich jak tworzenie wykresów.

`dplyr::bind_cols()` efektywnie łączy wiele ramek danych w kolumny.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Wizualizacja wyników modelowania

Teraz czas zobaczyć to wizualnie 📈. Stworzymy wykres punktowy wszystkich wartości `y` i `bmi` z zestawu testowego, a następnie użyjemy przewidywań, aby narysować linię w najbardziej odpowiednim miejscu, pomiędzy grupami danych modelu.

R oferuje kilka systemów do tworzenia wykresów, ale `ggplot2` jest jednym z najbardziej eleganckich i wszechstronnych. Umożliwia komponowanie wykresów poprzez **łączenie niezależnych komponentów**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Zastanów się chwilę, co tu się dzieje. Przez wiele małych punktów danych przebiega prosta linia, ale co dokładnie robi? Czy widzisz, jak można użyć tej linii, aby przewidzieć, gdzie nowy, niewidziany wcześniej punkt danych powinien pasować w odniesieniu do osi y wykresu? Spróbuj opisać praktyczne zastosowanie tego modelu.

Gratulacje, stworzyłeś swój pierwszy model regresji liniowej, dokonałeś za jego pomocą predykcji i wyświetliłeś ją na wykresie!



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
